## Insert Parameters
Parameters can be inserted automatically into a Copasi model from python code using PyCoTools

### Build a demonistration model
While antimony or the COPASI user interface are the preferred ways to build a model, PyCoTools does have a mechanism for constructing COPASI models. For variation and demonstration, this method is used here.

In [3]:
import os
import site
site.addsitedir('D:\pycotools3')
from pycotools3 import model, tasks, viz
## Choose a working directory for model
working_directory = os.path.abspath('')
copasi_file = os.path.join(working_directory, 'MichaelisMenten.cps')

if os.path.isfile(copasi_file):
    os.remove(copasi_file)

    
kf = 0.01
kb = 0.1
kcat = 0.05
with model.Build(copasi_file) as m:
    m.name = 'Michaelis-Menten'
    m.add('compartment', name='Cell')
    
    m.add('metabolite', name='P', concentration=0)
    m.add('metabolite', name='S', concentration=30)
    m.add('metabolite', name='E', concentration=10)
    m.add('metabolite', name='ES', concentration=0)
    
    m.add('reaction', name='S bind E', expression='S + E -> ES', rate_law='kf*S*E',
          parameter_values={'kf': kf})
    
    m.add('reaction', name='S unbind E', expression='ES -> S + E', rate_law='kb*ES',
         parameter_values={'kb': kb})

    m.add('reaction', name='ES produce P', expression='ES -> P + E', rate_law='kcat*ES', 
          parameter_values={'kcat': kcat})
    
mm = model.Model(copasi_file)
mm

Model(name=Michaelis-Menten, time_unit=s, volume_unit=ml, quantity_unit=mmol)

### Insert Parameters from Python Dictionary

In [4]:
params = {'E': 100,
          'P': 150}

## Insert into model 
I = model.InsertParameters(mm, parameter_dict=params)
I.parameters.index = ['Parameter Value']
I.parameters.transpose()

,Parameter Value
E,100
P,150


Alternatively use `inplace=True` argument to save automatically

In [5]:
I = model.InsertParameters(mm, parameter_dict=params, inplace=True)

### Insert Parameters from Pandas DataFrame

In [11]:
import pandas
params = {'(S bind E).kf': 50,
          '(S unbind E).kb': 96}
df = pandas.DataFrame(params, index=[0])
df

,(S bind E).kf,(S unbind E).kb
0,50,96


In [10]:
model.InsertParameters(mm, df=df, inplace=True)

PermissionError: [WinError 5] Access is denied: 'D:\\pycotools3\\docs\\source\\Tutorials\\MichaelisMenten.cps'

### Insert Parameters from Parameter Estimation Output
Get some parameter estimation data

In [13]:
## simulate data 
TC = tasks.TimeCourse(mm, end=100, step_size=1, intervals=100)

viz.Parse(TC).data.head()

,Time,S,ES,P,E
0,0,30.000000,0.0000,150.000,100.0000
1,1,0.740547,27.8340,151.425,72.1660
2,2,0.693069,26.5228,152.784,73.4772
3,3,0.649328,25.2720,154.079,74.7280
4,4,0.608948,24.0788,155.312,75.9212


In [15]:
config_dct = dict(
    models=dict(
        mm=dict(
            filename= mm.copasi_file
        )
    )
)

with tasks.ParameterEstimation.Context(context='s', parameters='a') as config:
    context.add_models(mm.copasi_file)
    context.add_experiments(TC.report_name)
    context.add_setting('working_directory', working_directory)
    
context.config

AttributeError: 'Context' object has no attribute 'config'

In [ ]:
## format copasi report headers
misc.format_timecourse_data(TC.report_name)

## run PEs
PE = tasks.ParameterEstimation(mm, TC.report_name, 
                                   method='genetic_algorithm', 
                                   population_size=10, 
                                   number_of_generations=10, 
                                   copy_number=2, 
                                   pe_number=2, 
                                   run_mode=True)
PE.write_config_file()
PE.setup()
PE.run()
print (PE.results_directory)

In [ ]:
##index=0 for best parameter set (i.e. lowest RSS)
model.InsertParameters(mm, parameter_path=PE.results_directory, index=0, inplace=True) 

### Insert Parameters using the `model.Model().insert_parameters` method
The same means of inserting parameters can be used from the model object itself

In [ ]:
mm.insert_parameters(parameter_dict=params, inplace=True)

### Change parameters using `model.Model().set`
Individual parameters can also be changed using the `set` method. For example, we could set the `metabolite` with name `S` `concentration` or `particle numbers` to `55`

In [ ]:
mm.set('metabolite', 'S', 55, 'name', 'concentration')

## or 

mm.set('metabolite', 'S', 55, 'name', 'particle_numbers')